In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
split_df = pd.read_pickle("split_df_vectorised.pkl")

In [151]:
df = pd.read_csv("XYZCorp_LendingData.txt", sep = "\t", low_memory = False)

In [9]:
df["purpose"] = df["purpose"].replace({"home_improvement":"house"})
df["purpose"].value_counts()


debt_consolidation    505392
credit_card           200144
house                  53469
other                  40949
major_purchase         16587
small_business          9785
car                     8593
medical                 8193
moving                  5160
vacation                4542
wedding                 2280
renewable_energy         549
educational              326
Name: purpose, dtype: int64

In [10]:
desc_df = df.dropna(subset = ["desc"])[["desc","purpose"]]

In [35]:
desc_sentences = desc_df["desc"].tolist()
desc_sentences = [re.sub(pattern = "(Borrower added on \d{2}\S\d{2}\S\d{2} >)", string = x,repl = "") for x in desc_sentences]
desc_sentences = [re.sub(pattern = "<br/>", string = x,repl = "<br>") for x in desc_sentences]

In [36]:
split_sentences = [x.lower() for x in desc_sentences]
split_sentences = [x.split("<br>") for x in desc_sentences]
split_sentences = [[y.strip() for y in x] for x in split_sentences]

split_sentences = [[y for y in x if y != ""] for x in split_sentences]
split_sentences = [[y.split(".") for y in x] for x in split_sentences]
split_sentences =  [[item for sublist in x for item in sublist] for x in split_sentences]

split_sentences = [[y.strip() for y in x] for x in split_sentences]
split_sentences = [[y for y in x if y != ""] for x in split_sentences]



In [37]:
split_sentences

[['I need to upgrade my business technologies'],
 ['I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I have finished college with an associates degree in business and its takingmeplaces'],
 ['to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace',
  "I'm very sorry to let my loan expired last time"],
 ['I plan on combining three large interest bills together and freeing up some extra each month to pay toward other bills',
  "I've always been a good payor but have found myself needing to make adjustments to my budget due to a medical scare",
  

In [108]:
for i in range(len(split_sentences)):
    if len(set(split_sentences[i])) != len(split_sentences[i]):
        split_sentences[i] = list(set(split_sentences[i]))
       

In [109]:
split_sentences[1]

['I plan to have it paid off as soon as possible/when i sell my old bike',
 'I plan to use this money to finance the motorcycle i am looking at',
 'I only need this money because the deal im looking at is to good to pass up',
 'I have finished college with an associates degree in business and its takingmeplaces']

In [110]:

split_df = desc_df[["desc","purpose"]].copy()
split_sentences = [[y.strip() for y in x] for x in split_sentences]
split_sentences = [[y for y in x if y != ""] for x in split_sentences]
split_df["desc"] = split_sentences

split_df = split_df.explode("desc")
split_df = split_df.dropna()
split_df = split_df.reset_index(drop = True)

In [111]:
split_df["desc"] = split_df["desc"].apply(lambda x:str(x).lower())
#all lower case, case doesn't give info for this task
split_df["desc"] = split_df["desc"].apply(lambda x:re.sub("[^A-Za-z']+", string = x, repl = " "))
#remove punctuation 
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))
split_df["desc_stop"] = split_df["desc"].apply(lambda x:[y for y in x.split(" ") if y not in en_stop])
#removed stopwords


In [84]:
split_df["desc"].apply(lambda x:[y for y in x.split(" ") if y not in en_stop])

0                   [need, upgrade, business, technologies]
1          [plan, use, money, finance, motorcycle, looking]
2             [plan, paid, soon, possible, sell, old, bike]
3              [need, money, deal, im, looking, good, pass]
4          [plan, use, money, finance, motorcycle, looking]
                                ...                        
341178    [senior, year, college, obtaining, bachelors, ...
341179    [qualify, financial, aid, used, stafford, loan...
341180                                    [tuition, approx]
341181                     [, month, courses, left, adds, ]
341182    [need, assistance, cover, tuition, graduation,...
Name: desc, Length: 341183, dtype: object

In [89]:
split_df["desc_stop"].values

array([list(['need', 'upgrade', 'business', 'technologies']),
       list(['plan', 'use', 'money', 'finance', 'motorcycle', 'looking']),
       list(['plan', 'paid', 'soon', 'possible', 'sell', 'old', 'bike']),
       ..., list(['tuition', 'approx']),
       list(['', 'month', 'courses', 'left', 'adds', '']),
       list(['need', 'assistance', 'cover', 'tuition', 'graduation', 'date', 'august', ''])],
      dtype=object)

In [8]:
#obtain word counts
from collections import Counter
cnt = Counter()
words = []
for text in split_df["desc_stop"].values:
    for word in text:
        words.append(word)
        cnt[word] += 1
one_list = [w for w in words if cnt[w] == 1]
#get list of words that only appear once


In [ ]:
df

In [251]:
split_df["desc_stop"].iloc[4]

['finished', 'college', 'associates', 'degree', 'business', 'takingmeplaces']

In [118]:
split_df["desc_rare"]  = split_df["desc_stop"].apply(lambda x:[y for y in x if y not in one_list])
#remove all words that only appear once


In [121]:
split_df.to_pickle("split_df_rare.pkl")

In [2]:
split_df = pd.read_pickle("split_df_rare.pkl")

In [138]:
cores  = multiprocessing.cpu_count()

In [3]:
import gensim.downloader

glove_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

KeyboardInterrupt: 

In [158]:
glove_vectors["credit"]

array([-0.036518  , -0.1251    , -0.056394  , -0.037371  , -0.03143   ,
       -0.057902  ,  0.012194  , -0.09384   ,  0.030637  , -0.0063236 ,
        0.0092002 , -0.032662  ,  0.051187  , -0.073442  , -0.080071  ,
        0.042418  ,  0.045727  , -0.017771  ,  0.054267  ,  0.0021157 ,
        0.040701  ,  0.062902  , -0.033553  ,  0.035752  , -0.056688  ,
       -0.027773  , -0.043931  ,  0.045159  ,  0.0086014 , -0.018957  ,
        0.02719   ,  0.038497  , -0.11996   , -0.062557  ,  0.0021302 ,
        0.015395  , -0.025056  , -0.0051397 , -0.043409  ,  0.057623  ,
       -0.018294  , -0.12969   ,  0.055268  ,  0.0085395 , -0.045727  ,
        0.0028698 ,  0.013547  , -0.021728  ,  0.016178  , -0.012503  ,
       -0.045338  ,  0.022954  ,  0.027089  ,  0.06132   , -0.077001  ,
        0.032047  ,  0.00059475, -0.024694  , -0.062178  , -0.028332  ,
       -0.028909  ,  0.023139  ,  0.16037   , -0.067911  , -0.032157  ,
        0.00038121, -0.078442  , -0.022071  , -0.0087877 , -0.04

In [161]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17988765583175778484
xla_global_id: -1
]


In [168]:
split_df["desc_stemmed"] = split_df["desc_stemmed"].apply(lambda x:[y for y in x if y != ""])

In [5]:
split_df["desc_rare"] =  split_df["desc_rare"].apply(lambda x:[y for y in x if y != ""])

In [6]:
def embed_words(sentence):
    vector = []
    counter = {}
    for word in sentence:
        counter[word] = cnt[word]
    sentence = sorted(sentence, key=counter.get, reverse = True)
    if len(sentence) > 20:
        sentence = sentence[:20]
    for word in sentence:
        try:
            vector.append(glove_vectors[word])
        except KeyError:
            pass
    return np.array(vector)

In [281]:
sentence = split_df["desc_rare"].iloc[0]
vector = []
counter = {}
for word in sentence:
    counter[word] = cnt[word]
sentence = sorted(sentence, key=counter.get, reverse = True)
sentence

['need', 'business', 'upgrade', 'technologies']

In [282]:
counter

{'need': 12399, 'upgrade': 421, 'business': 6134, 'technologies': 21}

In [9]:
split_df["desc_vectors"] = split_df["desc_rare"].apply(embed_words)

In [11]:
split_df.to_pickle("split_df_vectorised.pkl")

In [194]:
split_df["desc_rare"].explode().value_counts()

credit           93566
loan             74594
pay              67809
debt             62098
cards            45533
                 ...  
capri                1
anypayments          1
hopen                1
distributions        1
newroof              1
Name: desc_rare, Length: 15976, dtype: int64

In [285]:
 sorted([len(x) for x in split_df["desc_vectors"]], reverse = True)

[20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,


In [3]:
def padding(sentence):
    if len(sentence) < 20:
        diff = 20 - len(sentence)
        sentence = np.concatenate((sentence, np.zeros((int(diff),300))))
    return sentence

In [4]:
 padding(split_df["desc_vectors"].iloc[0])

array([[ 0.0037601, -0.0085634, -0.010759 , ...,  0.025789 , -0.052145 ,
        -0.042133 ],
       [-0.022481 , -0.045884 ,  0.0011983, ...,  0.0095043, -0.024917 ,
        -0.030083 ],
       [-0.014887 ,  0.066357 , -0.019932 , ..., -0.059062 ,  0.020761 ,
        -0.062035 ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [5]:
split_df = split_df[split_df['desc_rare'].map(lambda d: len(d)) > 0]
split_df = split_df[split_df['desc_vectors'].map(lambda d: len(d)) > 0]


In [6]:
split_df[split_df['desc_rare'].str.len() == 0]


,desc,purpose,desc_stop,desc_rare,desc_vectors


In [7]:
split_df["desc_vectors_pad"] = split_df["desc_vectors"].apply(padding)

In [8]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
split_df["purpose_enc"] = lb.fit_transform(split_df["purpose"].copy()) 

In [296]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
y_raw = ohe.fit_transform(split_df["purpose"].to_numpy().reshape(-1, 1)) 
y = pd.DataFrame(ohe.fit_transform(split_df["purpose"].to_numpy().reshape(-1, 1)), columns = ohe.categories_)

In [10]:
split_df["desc_vectors_pad"].tolist()

[array([[ 0.0037601, -0.0085634, -0.010759 , ...,  0.025789 , -0.052145 ,
         -0.042133 ],
        [-0.022481 , -0.045884 ,  0.0011983, ...,  0.0095043, -0.024917 ,
         -0.030083 ],
        [-0.014887 ,  0.066357 , -0.019932 , ..., -0.059062 ,  0.020761 ,
         -0.062035 ],
        ...,
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ]]),
 array([[ 0.076113 , -0.079565 ,  0.069583 , ...,  0.047581 , -0.10269  ,
         -0.053104 ],
        [ 0.10362  ,  0.017096 , -0.056124 , ..., -0.030689 , -0.042999 ,
         -0.039097 ],
        [ 0.019459 ,  0.042488 , -0.01285  , ..., -0.078292 ,  0.0017939,
         -0.073276 ],
        ...,
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.

In [11]:
split_df["purpose_enc"].astype("category").to_numpy()

array([1, 0, 0, ..., 8, 8, 8], dtype=int64)

In [115]:
split_df["desc_vectors_comb"] = split_df["desc_vectors"].apply(lambda x:np.add.reduce(x)) 
split_df.drop(["other","renewable_energy",""])
X = pd.DataFrame(split_df["desc_vectors_comb"].tolist()).to_numpy()

In [116]:
from sklearn.model_selection import train_test_split

y = split_df["purpose_enc"].astype("category")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size = 0.2)

In [90]:
X_train = X_train
y_train = y_train.to_numpy()

In [119]:
y_train.shape

(209848,)

In [120]:
import xgboost as xgb

model = xgb.XGBClassifier(objective = 'multi:softprob', num_class = 13)
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_class=13, num_parallel_tree=1,
              objective='multi:softprob', ...)

In [122]:
import pickle
with open("xgb_nlp.pkl","wb") as files:
    pickle.dump(model, files)

In [124]:
xgb_predict = model.predict(X_test)

In [128]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_test,xgb_predict))


0.5941016804416115


In [141]:
mapping = dict(zip(lb.classes_, range(1, len(lb.classes_)+1)))
list(mapping.keys())

['car',
 'credit_card',
 'debt_consolidation',
 'educational',
 'house',
 'major_purchase',
 'medical',
 'moving',
 'other',
 'renewable_energy',
 'small_business',
 'vacation',
 'wedding']

In [143]:
from sklearn.metrics import multilabel_confusion_matrix
conf = multilabel_confusion_matrix(y_test, xgb_predict)

In [138]:
conf

array([[[64015,   275],
        [ 1014,   274]],

       [[51752,  1189],
        [11493,  1144]],

       [[ 7503, 22170],
        [ 2158, 33747]],

       [[65300,    18],
        [  251,     9]],

       [[59891,  1124],
        [ 2634,  1929]],

       [[63238,   332],
        [ 1779,   229]],

       [[64851,    76],
        [  555,    96]],

       [[65001,    61],
        [  449,    67]],

       [[61293,   415],
        [ 3679,   191]],

       [[65463,     4],
        [  109,     2]],

       [[62308,   756],
        [ 1507,  1007]],

       [[65234,    42],
        [  261,    41]],

       [[64469,   156],
        [  729,   224]]], dtype=int64)

In [144]:
y_test, 

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12
purpose_enc,,,,,,,,,,,,,
0,274,8,837,0,18,84,0,0,29,0,37,1,0
1,15,1144,11180,3,135,9,10,8,41,0,76,5,11
2,88,1025,33747,6,481,63,30,20,137,3,244,11,50
3,0,3,216,9,3,0,0,0,9,0,20,0,0
4,11,36,2415,2,1929,35,2,5,47,1,77,1,2
5,95,19,1331,2,153,229,2,3,47,0,88,3,36
6,1,8,507,0,5,0,96,0,19,0,11,0,4
7,2,1,381,1,31,1,1,67,20,0,10,1,0
8,48,60,3039,3,176,82,28,21,191,0,164,15,43


In [374]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
import tensorflow.keras.backend as K



In [ ]:
model = Sequential()
model.add(Dense(60, input_shape = (20,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(13, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

In [ ]:

model.fit(X_train, y_train)

In [ ]:
set([x.shape for x in X_train ])

In [378]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 20) <dtype: 'float32'>
(None, 13) <dtype: 'float32'>
dense_2 (None, 20) float32
dense_3 (None, 60) float32
dropout (None, 15) float32
dense_4 (None, 15) float32


[None, None, None, None]

In [ ]:
X_train

In [22]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [18]:
import tensorflow_gpu
tensorflow_gpu.__version__

ModuleNotFoundError: No module named 'tensorflow_gpu'

In [24]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [56]:
from sklearn import datasets
iris = datasets.load_iris()

In [58]:
iris.data.shape

(150, 4)

In [59]:
iris.target.shape

(150,)

In [65]:
X_train.to_numpy().reshape((-1,20))

ValueError: cannot reshape array of size 209848 into shape (20)

In [70]:
split_df["desc_vectors_comb"] = split_df["desc_vectors"].apply(lambda x:np.add.reduce(x))

In [110]:
split_df["desc_vectors_comb"].explode()

0        -0.070309
0         0.002033
0        -0.039928
0        -0.060788
0        -0.152418
            ...   
334378   -0.000821
334378   -0.057447
334378   -0.017999
334378   -0.056753
334378   -0.320544
Name: desc_vectors_comb, Length: 98366700, dtype: object

In [153]:
df[df["purpose"] == "renewable_energy"]["desc"].tolist()

[nan,
 '  Borrower added on 11/29/11 > This Lending Club loan is to pay off $12,000 of a $15,000 higher interest rate 9.99% credit line for a solar system we financed.  We also have approx $11,000 no payment/no interest for 1 year credit line to finance the rest of the solar system that will be paid off when we receive the solar tax credit from filing our tax return in 2012. We also have a home improvement credit line from Lowes approx $10,000 that we pay $1,000 per month so that we can pay it off early.  Student loan balance approx $20,000 we pay $500 per month so that we can pay it off early. All other credit cards/lines of credit are paid off every month as we only use our credit cards to earn points.  <br>',
 nan,
 '  Borrower added on 11/16/11 > Ionizer purchase<br>',
 nan,
 '  Borrower added on 11/06/11 > I am trying to start life with a new outlook and want to consolidate a couple personal loans<br/>',
 nan,
 '  Borrower added on 10/22/11 > LOOKING TO PURCHASE HEALTHY WATER SYST